In [1]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()


--2019-03-24 19:01:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-03-24 19:01:32 (131 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [5]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
8/8 [==============================] - 3s 379ms/step - loss: 1.1849 - acc: 0.4875
Epoch 2/15
8/8 [==============================] - 0s 27ms/step - loss: 0.5624 - acc: 0.7125
Epoch 3/15
8/8 [==============================] - 0s 47ms/step - loss: 0.3045 - acc: 0.9000
Epoch 4/15
8/8 [==============================] - 0s 36ms/step - loss: 0.2300 - acc: 0.8875
Epoch 5/15
8/8 [==============================] - 0s 36ms/step - loss: 0.1260 - acc: 0.9625
Epoch 6/15
8/8 [==============================] - 0s 27ms/step - loss: 0.1545 - acc: 0.9000
Epoch 7/15
8/8 [==============================] - 0s 47ms/step - loss: 0.2051 - acc: 0.9250
Epoch 8/15
8/8 [==============================] - 0s 37ms/step - loss: 0.1038 - acc: 0.9750
Epoch 9/15
8/8 [==============================] - 0s 49ms/step - loss: 0.0653 - acc: 0.9875
Epoch 10/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0323 - acc: 1.0000
Reached 99.9% accuracy so cancellin